## 1. Загрузка данных

Настроим окружение: импортируем все необходимые библиотеки.

In [1]:
import pandas as pd
from IPython.display import display
import numpy as np
from numpy.linalg import inv
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

Сохраним в переменной data входные данные клиентов, которые необходимо защитить. Просмотрим первые 5 строк массива данных и информацию о типах данных признаков.

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
display(data.head())
display(data.info())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

Просмотрим уникальные значения целевого признака: количество страховых выплат клиенту за последние 5 лет.

In [4]:
display(data['Страховые выплаты'].value_counts())

0    4436
1     423
2     115
3      18
4       7
5       1
Name: Страховые выплаты, dtype: int64

## 2. Умножение матриц

Введем обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** При умножении матрицы признаков на обратимую матрицу качество линейной регрессии не изменится.

**Обоснование:** 

Допустим, что $X$ и $P$ - обратимые матрицы.

Рассчитаем вектор предсказаний $a'$ после умножения матрицы признаков $X$ на обратимую матрицу $P$. Распишем формулу обучения в предсказании для вектора предсказаний $a'$.
$$
a' = X P ((X P)^T X P)^{-1} (X P)^T y
$$

Исходя из свойства транспонированных матриц:

$$
(AB)^{T}=B^{T}A^{T}
$$

распишем выражение для транспонированного произведения матриц:

$$
a' = X P ((X P)^T X P)^{-1} (X P)^T y = X P (P^{T} X^{T} X P)^{-1} P^{T} X^{T} y
$$

На основе свойств обратных матриц для обратимых квадратных матриц и свойств единичной матрицы:

$$
(AB)^{{-1}}=B^{{-1}}A^{{-1}}
$$
$$
A A^{-1} = E
$$
$$
AE=EA=A
$$

преобразуем выражение:

$$
a' = X P ((X P)^T X P)^{-1} (X P)^T y = X P (P^{T} X^{T} X P)^{-1} P^{T} X^{T} y = X P P^{-1} (P^{T} X^{T} X)^{-1} P^{T} X^{T} y = X E (X^{T} X)^{-1} (P^{-1})^{T} P^{T} X^{T} y = X (X^{T} X)^{-1} (P^{-1} P)^{T} X^{T} y = X (X^{T} X)^{-1} E^{T} X^{T} y = X (X^{T} X)^{-1} X^{T} y = Xw
$$

В итоге получаем, что $a' = a$.

## 3. Алгоритм преобразования

**Алгоритм**

Для рещения задачи защиты данных клиентов страховой компании алгоритм преобразования будет содержать следующие шаги:
- создание переменных для признаков и целевого признака;
- обучение модели линейной регрессии на исходных признаках;
- определение коэффициента детерминации r2 для модели до преобразования признаков;
- создание случайной обратимой матрицы;
- умножение матрицы признаков на случайную обратимую матрицу;
- обучение модели линейной регрессии после преобразования признаков;
- определение коэффициента детерминации r2 для модели после преобразования;
- сравнение метрики предсказаний модели линейной регрессии до и после преобразования признаков.

**Обоснование**

На предыдущем этапе мы доказали, что при умножении матрицы признаков на обратимую матрицу качество предсказаний линейной регрессии не изменится. Запрограммируем предложенный алгоритм преобразований и подтвердим выдвинутое доказательство.

## 4. Проверка алгоритма

Создадим переменные для признаков и целевых признаков, приведем их к матричному типу атрибутом values.

In [5]:
features = data.drop('Страховые выплаты', axis=1).values
target = data['Страховые выплаты'].values

Определим размер матрицы признаков.

In [6]:
features.shape

(5000, 4)

Обучим модель линейной регрессии для исходной матрицы признаков и рассчитаем метрику R2.

In [7]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
initial_r2 = r2_score(target, predictions)

Создадим случайную квадратную матрицу $P$ размером, равным ширине матрицы признаков features. Выведем матрицу на экран.

In [8]:
matrix_P = np.random.rand(4,4)
display(matrix_P)

array([[0.15420363, 0.75352295, 0.70505824, 0.58211924],
       [0.0795655 , 0.88506067, 0.77001956, 0.32799892],
       [0.30879119, 0.48405765, 0.05255135, 0.31744872],
       [0.23749144, 0.77065356, 0.88968031, 0.83039584]])

Проверим обратимость матрицы $P$.

In [9]:
inv_matrix_P = inv(matrix_P)
display(inv_matrix_P)

array([[-22.19995933,   6.9183142 ,   3.78843315,  11.38154862],
       [  7.85683809,  -1.28085093,   0.3589444 ,  -5.13904951],
       [-11.65375795,   4.40097584,  -0.36901606,   6.57217295],
       [ 11.54331287,  -5.50509695,  -1.02124293,  -4.32290991]])

Умножим матрицу признаков features на обратимую матрицу $P$. Обучим модель линейной регрессии для преобразованной матрицы признаков и рассчитаем метрику R2.

In [10]:
features_transform = features.dot(matrix_P)
model.fit(features_transform, target)
predictions = model.predict(features_transform)
transform_r2 = r2_score(target, predictions)

Выведем на экран и сравним качества модели до и после преобразования.

In [11]:
print('Коэффициент детерминации R2 модели на исходных признаках:', initial_r2)
print('Коэффициент детерминации R2 модели на преобразованных признаках:', transform_r2)

Коэффициент детерминации R2 модели на исходных признаках: 0.42494550286668
Коэффициент детерминации R2 модели на преобразованных признаках: 0.4249455028666739


### Вывод:
Предложен алгоритм преобразования данных, благодаря которому сложно восстановить персональные данные клиентов страховой компании «Хоть потоп». Обоснована корректность его работы и показано, что после выполнения алгоритма преобразований качество модели линейной регрессии не ухудшилось. Коэффициент детерминации **R2** линейной регрессии до и после преобразование составляет **~0.425**.